In [ ]:
# Import der Bibliotheken 
import cv2
import csv
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from PIL import Image
from pillow_heif import register_heif_opener
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from scipy.spatial.distance import pdist, squareform

In [ ]:
###################
# SAM Settings
###################

# Select the path to the checkpoint file corresponding to the model
# Pfad zur Checkpointdatei passend zum Model auswählen 
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Change "\" to "\\" Example: C:\\Users\\User\\Desktop\\sam_vit_b_01ec64.pth

# Select the type of model: "vit_b", "vit_h", "vit_l"
# Modeltyp auswählen
model_type = "vit_b"

# Choose "cuda" if you have an Nvidia graphics card, otherwise choose "cpu"
# Wenn Nvidia-Grafikkarte "cuda", sonst "cpu" wählen
device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# SAM Automatic Mask Generator parameters
# Parameter der automatischen Maskengenerierung von SAM
mask_generator_Fungi = SamAutomaticMaskGenerator(
    model = sam, 
    points_per_side = 7, 
    pred_iou_thresh = 0.6, 
    stability_score_thresh = 0.6)


##########################################
# Data Input and Output Settings
##########################################

# Enable plotting of filters
# Filter Plotten lassen
plot_filter = True 
# Plot size
# Größe des Plots
plot_size = (10, 5) 


# Folder path to the images for analysis
# Ordnerpfad zu den Bildern für die Analyse 
folder_path = 'Bilder/einzel_Pilze/' # Change "\" to "\\"  Example: 'C:\\Users\\User\\Desktop\\Pictures\\'

# Enable saving outputs in a CSV file
# Speicherung der Ausgaben in einer CSV-Datei
csv_storage = True

# Save result images with masks
# Speicherung der Ergebnisbilder mit Masken
picture_saving = True

# Prefix for the folder name where the CSV file with the results is saved
# Prefix für den Ordnernamen, indem die CSV-Datei mit den Ergebnissen gespeichert wird
prefix = "Results"  # The execution timestamp will be added afterwards

# Number of decimal places for the results to be saved in the CSV file
# Nachkommastellen der Ergebnisse für die Abspeicherung in der CSV-Datei
decimals = 3

# Calculate the diameter of the mushrooms
# Berechnug des Durchmessers der Pilze
average_diameter = False

# Image cropping settings:
# Einstellung Bildcropping:
petri_cropping= True # True -> Automatic cropping of the petri dish is active

# Maximum target size of the image in pixels as the basis for petri dish cropping
# Maximale Zielgröße des Bildes in Pixel als Basis für das Petrischalen Croppen
max_width = 600
max_height = 600

# Scale in px/unit
# Maßstab in px/Einheit
scale=1
# True if automatically calculated scale should be used
# True, wenn automatisch berechneter Maßstab verwendet werden soll
auto_scale= False
# Petri dish outer diameter
# Größe Petrischale Außendurchmesser 
norm_d_petri = 500 

######################################################################################
# Filter Settings -> Activate the Filter with True / Deactivate the Filter with False
######################################################################################

# Automatically activate Petri filter when Petri dish cropping is active
# Filter Petri automatisch an, wenn Petrischalen-Cropping aktiv ist
rim = 1 # Border thickness in px

# Filter only masks in the center of the image 
# Filter nur Masken im Bereich des Zentrums des Bildes 
filter_center = True

# Filter bbox to area of masks ratio too large 
# Filter bbox zu Fläche der Masken Verhältnis zu groß 
filter_bbox_to_big = True
max_bbox_to_area_ratio = 1.8 

# Percentage deviation of bbox coordinates for Similar Filter
# Prozentuale Abweichung der bbox-Koordinaten für Similar Filter
filter_similar_segmentation = True
similar_percentage = 0.04

# Filter to detect the largest mushroom in nested masks
# Filter zur Erkennung des größten Pilzes in verschachtelten Masken
filter_mask_in_mask= True

# Percentage brightness difference for brightness difference filter to background
# Prozentuale Differenz Helligkeit für Filter nach Helligkeitsdifferenz zu Hintergrund
filter_brightness_difference = False
brightness_percentage = 0.15

# Filtering masks with too small area
# Filterung Masken mit zu kleiner Fläche
filter_small_area = False
area_percentage= 0.06

# Mask with highest predicted_iou will be taken. Only for single mushrooms!
# Maske mit höchstem predicted_iou wird genommen. Nur für einzel Pilze!
filter_highest_predicted_iou = True

In [ ]:
#Antwortausgabe mit Box und Maske im Bild
def show_annsbox(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    # Erstellt ein leeres Bild-Array mit Alpha-Kanal für die Transparenz
    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0

    # Durchläuft jede Annotation
    for ann in sorted_anns:
        m = ann['segmentation']  # Maske
        color_mask = np.concatenate([np.random.random(3), [0.35]])  # Zufällige Farbe für die Maske

        # Färbt die Maske im Bild-Array
        img[m] = color_mask

        # Zeichnet die Bounding Box
        bbox = ann['bbox']
        x, y, w, h = bbox
        rect = plt.Rectangle((x, y), w, h, fill=False, edgecolor=color_mask[:3], linewidth=2)
        ax.add_patch(rect)

    # Zeigt das Bild mit Masken und Bounding Boxes
    ax.imshow(img)

In [ ]:
# Funktionen der Filter 

# Überprüfung, ob Pixel sich im Kreis befindet
def pixel_in_circle(x, y, center_x, center_y, radius):
    return (x - center_x)**2 + (y - center_y)**2 <= radius**2

# Filterung nach Masken die sich innerhalb der Petrischale befinden
def filter_masks_by_circle(masks):
    center = (cropped_image.shape[1] / 2, cropped_image.shape[0] / 2)
    radius = radius_petri - rim # Außenradius der Petrischale - Rand
    filtered_masks = [
        mask for mask in masks 
        if all(
            pixel_in_circle(x_rel, y_rel, *center, radius)
            for y_rel, row in enumerate(mask['segmentation'])
            for x_rel, pixel in enumerate(row) if pixel
        )
    ]
    return filtered_masks 


# Filterung nach Masken zu BBox Verhältnis
def filter_wrong_bbox(new_masks):
    filtered_masks = []
    for mask in new_masks:
        bbox = mask['bbox']
        mask_area = mask['area']
        
        # Berechnung der Fläche der BBox
        bbox_area = bbox[2] * bbox[3]
        
        # Filterung der Masken mit zu großem Verhältnis
        if 1 < bbox_area / mask_area < max_bbox_to_area_ratio:
            filtered_masks.append(mask)
        
    return filtered_masks


# Filterung von Masken mit änlicher BBox
def is_bbox_similar(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2

    image_height, image_width = cropped_image.shape[:2]

    # Berechnen des Schwellenwertes basierend auf dem Prozentsatz der Bildbreite und -höhe
    threshold = min(image_width, image_height) * similar_percentage

    xdiff=abs(x1-x2) # Differenz x-Koordinate links oben
    ydiff=abs(y1-y2) # Differenz y-Koordinate links oben
    xrdiff=abs((x1+w1)-(x2+w2)) #Differenz x-Koordinate rechts unten
    yrdiff=abs((y1-h1)-(y2-h2)) #Differenz y-Koordinate rechts unten

    return (xdiff < threshold and ydiff < threshold) or (xrdiff < threshold and ydiff < threshold) or (yrdiff < threshold and xdiff < threshold)

# Filterung der ähnlichen BBoxen nach höherem predicted_iou
def filter_similar_bboxes(new_masks):
    filtered_masks = []
    for mask1 in new_masks:
        bbox1 = mask1['bbox']
        max_iou_mask = mask1
        max_iou = mask1['predicted_iou']
        for mask2 in new_masks:
            bbox2 = mask2['bbox']
            # Überprüft, ob die BBoxen verschieden und ähnlich sind
            if bbox1 != bbox2 and is_bbox_similar(bbox1, bbox2):
                # Überprüft, ob die IOU der anderen Maske höher ist \
                # als der aktuelle maximale IOU
                if mask2['predicted_iou'] > max_iou:
                    max_iou = mask2['predicted_iou']
                    max_iou_mask = mask2
        # Überprüft, ob die BBoxen bereits in der gefilterten Liste vorhanden ist
        is_duplicate = False
        for filtered_mask in filtered_masks:
            if filtered_mask['bbox'] == max_iou_mask['bbox']:
                is_duplicate = True
                break
        # Wenn die BBox nicht in der Liste vorhanden ist, füge sie hinzu
        if not is_duplicate:
            filtered_masks.append(max_iou_mask)
    return filtered_masks


# Filterung der Masken die sich in dem Zentrum des Bildes befinden
def filter_masks_by_center(masks):
    filtered_masks = []
    # Berechnung Bildmittelpunkt
    image_center_x = cropped_image.shape[1]/ 2
    image_center_y = cropped_image.shape[0]/ 2

    for mask in masks:
        bbox = mask['bbox']
        # Überprüft, ob der Mittelpunkt des Bildes innerhalb der BBox ist 
        if ((bbox[0] <= image_center_x <= bbox[0] + bbox[2]) 
            and (bbox[1] <= image_center_y <= bbox[1] + bbox[3])):
            filtered_masks.append(mask)

    return filtered_masks


# Bedingung ob sich BBoxen innerhalb von anderen BBoxen befinden
def is_bbox_enclosed(bbox1, bbox2):
    # Extrahiere die Koordinaten der BBoxen
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2

    # Überprüft, ob bbox1 komplett von bbox2 umschlossen wird
    return (x2 <= x1 and y2 <= y1 and (x2 + w2) >= (x1 + w1) and (y2 + h2) >= (y1 + h1))

# Filterung der verschachtelten BBoxen nach der Größten BBox
def filter_biggest_bbox(new_masks):
    filtered_masks = []
    for mask1 in new_masks:
        bbox1 = mask1['bbox']
        is_contained = False
        for mask2 in new_masks:
            bbox2 = mask2['bbox']
            # Überprüft, ob bbox1 von bbox2 umschlossen ist
            if bbox1 != bbox2 and is_bbox_enclosed(bbox1, bbox2):
                is_contained = True # Markierung das bbox umschlossen ist 
                break
        # Überprüft, ob die bbox von anderer Maske umschlossen ist
        if not is_contained:
            filtered_masks.append(mask1)
    return filtered_masks


# Filterung nach der Maske mit höchstem predicted_iou Wert 
def filter_masks_by_highest_predicted_iou(masks):
    mask_with_highest_iou = max(masks, key=lambda mask: 
                                mask['predicted_iou']) 
    return [mask_with_highest_iou]


# Erstellung einer Maske für den Hintergrund und Berechnung dessen Farbwerte 
def calculate_background_color(masks):
    # Erstellung einer Kopie der ersten Maske aus der Liste
    combined_mask = copy.deepcopy(masks[0]['segmentation'])

    # Kombination aller Masken
    for mask in masks[1:]:
        combined_mask = np.logical_or(combined_mask, mask['segmentation'])

    # Invertierung der Maske
    background_mask_full = ~combined_mask

    # Schleife durch jeden Pixel in der Maske 
    for x in range(len(background_mask_full)):
        for y in range(len(background_mask_full[0])):
            # Überprüft, ob der Pixel im Kreis liegt und nicht \
            # von den kombinierten Masken erfasst wird
            background_mask_full[x, y] = background_mask_full[x, y] and pixel_in_circle(x, y, cropped_image.shape[1]/2, cropped_image.shape[1]/2, radius_petri-rim)

    # Überprüft, ob die Formen von background_mask_full \
    # und cropped_image übereinstimmen
    if background_mask_full.shape == cropped_image.shape[:2]:
        # Extrahiert die Hintergrundfarbe und berechnet den Durchschnitt
        true_pixel_colors = cropped_image[background_mask_full]
        background_color_mean = np.mean(true_pixel_colors, axis=0)

        return background_color_mean
    else:
        # Fehlermeldung, wenn die Formen nicht übereinstimmen
        print("Error in background color calculation")
        return 0


#Filterung nach der Differenz des Mittels des Farbwertes von Maske zu Hintergrund 
def filter_avg_brightnessdifference(masks):
    filtered_masks = []

    # Berechnung des größten Helligkeitsunterschieds
    max_brightness_difference = max([abs(np.mean(np.mean(cropped_image[mask['segmentation']], axis=0) - background_color_mean)) for mask in masks])

    # Prozentualer Schwellenwert basierend auf \
    # dem größten Helligkeitsunterschied
    threshold_brightness = max_brightness_difference * brightness_percentage

    for mask in masks:
        # Extrahiert die Pixel in der Maske und berechnet \
        # den durchschnittlichen Farbwert
        pixel_in_mask = cropped_image[mask['segmentation']]
        avg_color = np.mean(pixel_in_mask, axis=0)

        # Berechne den Helligkeitsunterschied zum Hintergrund
        brightness_difference = abs(np.mean(avg_color - background_color_mean))
        # Überprüfe, ob der Helligkeitsunterschied \
        # den Schwellenwert überschreitet
        if brightness_difference > threshold_brightness:
            filtered_masks.append(mask)

    return filtered_masks,threshold_brightness

# Filterrung zu kleiner Flächen
def filter_area(masks):
    filtered_masks = []
    image_height, image_width = cropped_image.shape[:2]
    # Berechnen des Schwellenwertes basierend auf dem Prozentsatz der Bildfläche
    threshold = image_width* image_height * area_percentage
    for mask in masks:
        if mask['area'] > threshold:
            filtered_masks.append(mask)
    return filtered_masks
    

In [ ]:
# Funktionen zur Auswertung der Masken

# Berechnung der Parameter der Pilze und Abspeicherung in neue Dictonaries
def extract_fungi(masks,scale_factor,threshold_brightness):
    new_list = [] #Liste der Informationen über Pilze initialisieren
    i=0
    # Sortiere die Liste nach dem Schlüssel 'area' absteigend, um den größten Pilz zuerst zu haben
    masks.sort(key=lambda x: x['area'], reverse=True)
    for mask in masks:
        i+=1
        # Extrahieren der Segmentationsmaske, bbox und area der Masken
        area = mask['area'] * scale_factor * scale_factor
        bbox = mask['bbox']
        segmentation = mask['segmentation']

        #Berechnung der Parameter
        spread = [bbox[2] * scale_factor, bbox[3] * scale_factor] 
        position = calculate_fungus_position(segmentation) * scale_factor
        roundness, perimeter = calculate_roundness(segmentation)
        if average_diameter: # Sehr Rechenintensive aber dafür genauere Berechnung des Durchmessers
            diameter = calculate_average_diameter(segmentation) * scale_factor 
        else:
            diameter = (perimeter / np.pi) * scale_factor 
        color, hex_color_mean = calculate_color(segmentation)
        brightness_difference = (abs(np.mean(color - background_color_mean)))
        bbox_to_area_ratio = (bbox[2] * bbox[3])/mask['area']

        # Die berechneten Informationen über den Pilz als Dictonary zur Liste hinzufügen
        new_list.append({
            'Nr' : count,  # Nummer des Bildes
            'filename' : image_file,  # Dateiname des Bildes
            'fungus' : i,  # Nummer des Pilzes 
            'position': position.tolist(),  # Position Mittelpunkt des Pilzes in [x,y]
            'area': area,  # Fläche des Pilzes
            'spread': spread,  # Ausbreitung in [x,y] des Pilzes
            'roundness': roundness,  # Rundheit des Pilzes
            'perimeter': perimeter * scale_factor,  # Umfang des Pilzes
            'diameter': diameter,  # Durchmesser des Pilzes
            'color_RGB': color.tolist(),  # Farbe des Pilzes im RGB-Format
            'color_hexadecimal': hex_color_mean,  # Farbe im Hexadezimalformat
            # Helligkeitsunterschied des Pilzes zu Hintergrund
            'brightness_difference' : brightness_difference,  
            # Helligkeitsschwellen des Filters
            'threshold_brightness' : threshold_brightness, 
            # Verhältnis der Begrenzungsbox zur Fläche des Pilzes
            'bbox_to_area_ratio' : bbox_to_area_ratio  
        })

    return new_list



# Berechung der gewichten Schwerpunkts des Pilze für Positionsbestimmung im Koordinaten-System
def calculate_fungus_position(segmentation):
    # Berechnet die Koordinaten aller Pixel, die zum Pilz gehören
    fungus_pixels = np.transpose(np.nonzero(segmentation))

    # Berechnet den Schwerpunkt als Mittelwert der x- und y-Koordinaten
    fungus_position = np.mean(fungus_pixels, axis=0)

    return fungus_position

# Berechnung der Farbwerte des Pilzes
def calculate_color(segmentation):
    # Extrahiert Farben der Pixel, die zum Pilz gehören
    true_pixel_colors = cropped_image[segmentation]

    # Berechnet den mittleren Farbwert des Pilzes
    color_mean = np.mean(true_pixel_colors, axis=0)

    # Konvertiert in das Hexadezimalformat mit führendem # und Nullen für zweistellige Werte
    hex_color_mean = "#{:02x}{:02x}{:02x}".format(*[int(c) for 
                                                    c in color_mean])

    return color_mean, hex_color_mean

# Berechnung der Rundheit und des Umfangs des Pilzes
def calculate_roundness(segmentation):
    # Berechnet Flächeninhalt des Objekts
    area = np.sum(segmentation)

    # Berechnet Umfang des Objekts (Anzahl der Pixel im Rand des Objekts)
    edge = segmentation ^ np.roll(segmentation, 1, axis=0)
    edge += segmentation ^ np.roll(segmentation, 1, axis=1)
    perimeter = np.sum(edge)

    # Berechnet Rundheit (Wert zwischen 1 und 0)
    roundness = (4 * np.pi * area) / (perimeter**2)

    return roundness, perimeter

# Berechnung des Durchschnittlichen Durchmesseres als mittel der maximalen Distanzen jedes Punktes, Rechnungsintensiv!!
def calculate_average_diameter(segmentation):
    # Findet Koordinaten aller Pixel, die zum Pilz gehören
    fungus_pixels = np.transpose(np.nonzero(segmentation))
        
    # Berechnet paarweise euklidische Distanzen zwischen den Pixeln
    distances = squareform(pdist(fungus_pixels))

    # Berechnet den gemittelten Durchmesser als Durchschnitt \
    # der maximalen Distanzen für jeden Punkt
    average_diameter = np.mean(np.max(distances, axis=0))

    return average_diameter

In [ ]:
# Funktion zum Runden auf beliebige Nachkommastellen
def round_value(value):
    if isinstance(value, float):
        return round(value, decimals)
    elif isinstance(value, list):
        return [round(v, decimals) if isinstance(v, float) else v for v in value]
    else:
        return value

# Speicherung der Listen in eine CSV-Datei
def save_list_of_dicts_to_csv(data_list, folder_path, filename):
    # Überprüfen, ob die Datenliste leer ist
    if not data_list:
        print("Die Datenliste ist leer. Es wurden keine Daten in die CSV-Datei geschrieben.")
        return

    # Dateipfad für die CSV-Datei
    filepath = os.path.join(folder_path, filename)

    # Öffnet die CSV-Datei im Anhängemodus, falls vorhanden, oder erstellt eine neue Datei
    mode = 'a' if os.path.exists(filepath) else 'w'
    with open(filepath, mode, newline='') as csvfile:
        # Definiert die Feldnamen für den CSV-Header basierend auf dem ersten Dictionary in der Liste
        fieldnames = data_list[0].keys()

        # Erstelle einen CSV-Writer mit Semikolon als Trennzeichen
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')

        # Falls die Datei neu erstellt wurde, schreibe den Header
        if mode == 'w':
            writer.writeheader()

        # Schreibt die Daten aus der Liste von Dictionaries in die CSV-Datei
        for data_dict in data_list:
            # Begrenzt die Anzahl der Nachkommastellen auf den übergebenen Wert
            data_dict_rounded = {key: round_value(value) for key, value in data_dict.items()}
            writer.writerow(data_dict_rounded)

    print(f'Die Daten wurden erfolgreich in die CSV-Datei "{filename}" im Ordner "{folder_path}" gespeichert.')


def save_masked_image(Filter, image_name):
    # Überprüft die Dateierweiterung
    if '.' in image_name:
        # Trennt den Dateinamen und die Erweiterung
        filename, extension = os.path.splitext(image_name)
        # Ersetzt die Erweiterung durch '.jpg'
        image_name = filename + '.jpg'
    else:
        # Fügt '.jpg' als Erweiterung hinzu
        image_name += '.jpg'

    # Passt den Ausgabepfad entsprechend an
    output_path = os.path.join(csv_folder_path, image_name)

    # Plot des Bildes mit den Masken und Speicherung der Ausgabe
    plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    show_annsbox(Filter)
    plt.savefig(output_path)
    plt.close()

In [ ]:
# Registrieren des HEIF-Openers
register_heif_opener()

# Liste der unterstützten Bildformate
supported_formats = ['.jpg', '.jpeg', '.png','.heic','.jfif','.webp']

# Eine Liste aller Dateien im Ordner
image_files = os.listdir(folder_path)

# Name des Ordners für die CSV-Dateien basierend auf dem aktuellen Zeitstempel
csv_folder_name = f"{prefix}_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Pfad zum Ordner, in dem die CSV-Dateien gespeichert werden sollen
csv_folder_path = os.path.join(os.path.dirname(folder_path), csv_folder_name)

if csv_storage or picture_saving:
    # Erstelle den Ordner für die CSV-Dateien
    os.makedirs(csv_folder_path, exist_ok=True)

#Startzeit für die Laufzeitberechnung
start_time = time.time()

count=0 #Initialisierung des Durchlaufzählers 
index_no_condition = []
# Iteration über alle Bilder im Ordner
for image_file in image_files:

    # Pfad zum aktuellen Bild
    image_path = os.path.join(folder_path, image_file)
    # Überprüfung, ob es sich um eine Datei handelt und ob die Dateiendung ein unterstütztes Bildformat ist
    if os.path.isfile(image_path):

        # Überprüfung, ob die Dateiendung ein unterstütztes Bildformat ist
        file_extension = os.path.splitext(image_file)[1].lower()
        if file_extension in supported_formats:

            # Laden des Bildes basierend auf dem Dateiformat
            if file_extension == '.heic':

                image = Image.open(image_path)
                # Konvertieren in ein OpenCV-Bild
                src = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            else:
                src = cv2.imread(image_path, cv2.IMREAD_COLOR)

            count+=1 #Zählen der Durchläufe
            # Zahl des Durchlaufes und Name der Bilddatei  
            print(f"Bilddatei {count}: {image_file}")

            # Erstellung des Namens für die CSV-Datei
            csv_filename = os.path.splitext(image_file)[0] + '.csv'
            
            if petri_cropping: # Kontrolliert ob das Petrischalen Cropping erwünscht ist

                # Auswahl und Berechnung des kleineren Verkleinerungsfaktors
                scaling_factor = min(max_width / src.shape[1], max_height / src.shape[0]) 

                # Neue Größe berechnen
                new_width = int(src.shape[1] * scaling_factor)
                new_height = int(src.shape[0] * scaling_factor)

                # Bild skalieren, in Graustufen umgewandeln und filtern
                src_resized = cv2.resize(src, (new_width, new_height), interpolation=cv2.INTER_AREA)
                gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY) 
                gray = cv2.medianBlur(gray, 5) # Graustufenbild filtern

                # Erstellung Kreis mithilfe Hough Circle Transformation
                circle = cv2.HoughCircles(
                    gray, cv2.HOUGH_GRADIENT, 1, minDist=100000,
                    param1=250, param2=25,
                    minRadius=75, maxRadius=100000)
                        
                # Kreiserstellung
                if circle is not None:
                    center_x, center_y, radius_petri = np.round(circle[0, 0]).astype(int)


                # Erstellung der Maske
                mask = np.zeros(src_resized.shape[:2], dtype="uint8") # Leere Maske
                cv2.circle(mask, (center_x, center_y), radius_petri, 255, -1)
                masked = cv2.bitwise_and(src_resized, src_resized, mask=mask)


                # Findet Begrenzungsrahmen um die Maske
                x, y, w, h = cv2.boundingRect(mask) 

                # Zuschneiden des Bildes basierend auf dem Begrenzungsrahmen der Maske
                cropped_image = masked[y:y+h, x:x+w]


                # Berechnung des angenommenen Durchmessers der Petrischale in px
                diameter_petri = radius_petri * 2

                # Maßstab für die Umwandlung Pixel in reale Einheit 
                # norm_d_petri ist der genormte Durchmesser der Petrischale in der Einheit
                scale_automatic = (norm_d_petri/diameter_petri) if diameter_petri != 0 else 1 #Einheit/px 

                # Bedingung ob automatisch berechneter Maßstab verwendet werden soll
                if auto_scale:
                    scale=scale_automatic
            else:
                cropped_image=src

            
            # Automatische Generierung der Masken mit SAM
            masks = mask_generator_Fungi.generate(cropped_image)

            # Beginn der Filterung

            # Filterung nach Bereich innerhalb der Petrischale, falls Petrischale cropping an ist
            if petri_cropping:
                Filter1 = filter_masks_by_circle(masks)
            else:
                Filter1=masks
                


            if len(Filter1) > 0: #Kontrolle ob es masken innerhalb der Petrischale gibt, wenn nicht Error und Abbruch für das Bild da keine Masken von sam gefunden wurden
            
                # Berechnung der Hintergrundfarbe
                background_color_mean= calculate_background_color(Filter1)

                # Filterung nach Zentrum der Petrischale
                if filter_center:
                    Filter2=filter_masks_by_center(Filter1)
                    if len(Filter2)<1: 
                        Filter2=Filter1
                else:
                    Filter2=Filter1

                # Filterung nach Größe von Bboxen im Verhätlnis zur Maske
                if filter_bbox_to_big:
                    Filter3=filter_wrong_bbox(Filter2)
                    if len(Filter3)<1: 
                        Filter3=Filter2
                else:
                    Filter3=Filter2

                # Filterung nach ähnlichen Bboxen
                if filter_similar_segmentation:
                    Filter4=filter_similar_bboxes(Filter3)
                    if len(Filter4)<1: 
                        Filter4=Filter3
                else:
                    Filter4=Filter3

                # Filterung nach der Verschachtelungen der Maske  
                if filter_mask_in_mask:
                    Filter5=filter_biggest_bbox(Filter4)
                    if len(Filter5)<1: 
                        Filter5=Filter4
                else:
                    Filter5=Filter4

                # Filterung nach Helligkeitsdiffernez zum Hintergrund
                if filter_brightness_difference :
                    Filter6,threshold_brightness=filter_avg_brightnessdifference(Filter5)
                    if len(Filter6)<1: 
                        Filter6=Filter5
                else:
                    Filter6=Filter5
                    threshold_brightness=0
                    background_color_mean=0
                
                #Filterung nach Fläche
                if filter_small_area:
                    Filter7=filter_area(Filter6)
                    if len(Filter7)<1: 
                        Filter7=Filter6
                else:
                    Filter7=Filter6

                # Filterung nach höchstem predictet_iuo
                if filter_highest_predicted_iou:
                    Filter8=filter_masks_by_highest_predicted_iou(Filter7)
                    if len(Filter8)<1: 
                        Filter8=Filter7
                else:
                    Filter8=Filter7                

                # Berechnung der Parameter der Pilze
                Fungi=extract_fungi(Filter8,scale,threshold_brightness) # Falls Größe der Petrischale (norm_d_petri) Maßstab=1   
                
                if csv_storage:
                    # Speicherung der Parameter in einer CSV-Datei mit Namen des Bildes 
                    save_list_of_dicts_to_csv(Fungi,csv_folder_path,'Results.csv') 

                if picture_saving:
                    # Speicherung der Bilder mit Masken im Results ordner
                    save_masked_image(Filter8, f"filtered_{image_file}")
                    save_masked_image(masks, f"og_{image_file}")


                if plot_filter:
                    # Ploten der Filter
                    plt.figure(figsize=plot_size)  # Größe des Plots

                    plot_count = 1  # Zähler für die Anzahl der geplotteten Filter

                    # Filterung nach Zentrum der Petrischale
                    if petri_cropping:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter1)
                        plt.title('filter masks in Circle')
                        plot_count += 1

                    # Filterung nach Zentrum der Petrischale
                    if filter_center:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter2)
                        plt.title('filter masks in Center')
                        plot_count += 1

                    # Filterung nach Größe von Bboxen im Verhätlnis zur Maske
                    if filter_bbox_to_big :
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter3)
                        plt.title('filter wrong bbox')
                        plot_count += 1

                    # Filterung nach ähnlichen Bboxen
                    if filter_similar_segmentation:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter4)
                        plt.title('filter similar bbox')
                        plot_count += 1

                    # Filterung nach der Verschachtelungen der Maske
                    if filter_mask_in_mask:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter5)
                        plt.title('filter bbox in bbox')
                        plot_count += 1

                    # Filterung nach Helligkeitsdiffernez zum Hintergrund
                    if filter_brightness_difference:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter6)
                        plt.title('filter brightness difference')
                        plot_count += 1

                    #Filterung nach Fläche
                    if filter_small_area:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter7)
                        plt.title('filter small area')
                        plot_count += 1

                    # Filterung nach höchstem predictet_iuo
                    if filter_highest_predicted_iou:
                        plt.subplot(2, 4, plot_count)
                        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                        show_annsbox(Filter8)
                        plt.title('filter highest predicted_iou')
                        plot_count += 1

                    plt.tight_layout()
                    plt.show()

            else: # Schleife wenn keine Maske nach dem Filter nach dem Petrischalenbereich gefunden wurden 
                # Plotten der Orginal Masken und des 1. Filters 
                plt.figure(figsize=(6, 6))

                plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                show_annsbox(masks)
                plt.title('ERROR')
                    
            
# Berechnen der Laufzeit
end_time = time.time()
laufzeit = end_time - start_time     

# Ausgabe der Laufzeit und der Druchläufe 
print(f"Anzahl der Durchläufe: {count} Bilder")
print(f"Laufzeit insgesamt: {laufzeit/60} Min")
print(f"Durchschnittliche Laufzeit pro Bild: {laufzeit/count} sec")
